The code is identical to: https://osf.io/9de3m/?view_only=214a0d84044644a1a7405308e18b7df1
, which is explained in the following publication: "From Embeddings to Explainability: A Tutorial on Transformer-Based Text Analysis for Social and Behavioral Scientists", see: https://osf.io/preprints/psyarxiv/bc56a

Only the end of the code ("Julius idea") is written by myself

When running this notebook in Google Colab, each cell can be run by pressing the "Play" button. We start with installing all packages. Error messages can be ignored.

In [ ]:
#!pip install accelerate
#!pip install datasets
#!pip install transformers
#!pip install gdown

We load all packages that are necessary for the first steps of the analysis.

In [ ]:
import datasets
from datasets import load_dataset
import transformers
import accelerate
import random
import torch
from transformers import set_seed
import gdown

We are setting the random seed. This step ensures that all results are reproducible when repeatedly running the code.

In [ ]:
def set_gen_seed(seed):
    set_seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_gen_seed(42)

In the following cell, we get the files train.csv and test.csv from a public Google Drive repository. These files are also provided in the OSF repository mentioned in the main text. If these files are stored on your local computer, you can also upload them from there. In Google Colab, this is done via the following steps:
1.  Select the folder symbol on the left.
2.  Click on the button for uploading files. It has a file symbol, with an arrow pointing up.
3. Select the two files "test.csv" and "train.csv" from your hard drive and upload them.

If you complete these steps successfully, the two files should be visible among the files available in this workspace.

In [4]:
# URLs to the files in Google Drive
train_file_url = 'https://drive.google.com/uc?id=1IpcfeM-susWq8j1ysrdLe9dup_0kzz74'
test_file_url = 'https://drive.google.com/uc?id=1LmT2cgb8mp4agot82gU4cysKF_zdzY4o'

# Download the files
gdown.download(train_file_url, 'train.csv', quiet=False)
gdown.download(test_file_url, 'test.csv', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1IpcfeM-susWq8j1ysrdLe9dup_0kzz74
To: /home/fenn/Desktop/Workshop LLMs/4_textClassification/train.csv
100%|██████████| 20.5k/20.5k [00:00<00:00, 4.80MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LmT2cgb8mp4agot82gU4cysKF_zdzY4o
To: /home/fenn/Desktop/Workshop LLMs/4_textClassification/test.csv
100%|██████████| 5.10k/5.10k [00:00<00:00, 12.7MB/s]


'test.csv'

In the next cell, we are loading the data files.

In [5]:
dataset = load_dataset("csv", data_files = {'train': 'train.csv', 'test': 'test.csv'}, sep =";", names = ["text","label"])

We take a look at two example data, namely the first entry of the training data set (indexed 0) and the second entry of the test data set (indexed 1).

In [6]:
dataset["train"][[0]]

{'text': ["I'm kind of down right now because I was late to my chemistry class this morning. So I'm feeling really sad about that because I missed my homework submission, and now I'm just trying to get over it and sitting at home."],
 'label': [0]}

In [7]:
dataset["test"][[1]]

{'text': ["Right now, I am at PLACE with NAME. We're waiting for our order. I'm feeling really, really hungry because your boy here didn't want to get up, so we didn't have breakfast and I'm also really happy and excited for what the day has in store."],
 'label': [1]}

We load the DistilBERT model and apply tokenization:

In [8]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [9]:
def tokenize(batch):
  return tokenizer(batch["text"], padding = True, truncation = True)

In [10]:
dataset_encoded = dataset.map(tokenize, batched = True, batch_size= None)

After tokenization, we take a look at the dataset again. Via the output of the print function, we see that there are new columns, namely "input_ids" and "attention_mask".

In [11]:
print(dataset_encoded["train"])

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 80
})


We take a look at one example, the first entry of the training set.

In [12]:
dataset_encoded["train"][0]

{'text': "I'm kind of down right now because I was late to my chemistry class this morning. So I'm feeling really sad about that because I missed my homework submission, and now I'm just trying to get over it and sitting at home.",
 'label': 0,
 'input_ids': [101,
  1045,
  1005,
  1049,
  2785,
  1997,
  2091,
  2157,
  2085,
  2138,
  1045,
  2001,
  2397,
  2000,
  2026,
  6370,
  2465,
  2023,
  2851,
  1012,
  2061,
  1045,
  1005,
  1049,
  3110,
  2428,
  6517,
  2055,
  2008,
  2138,
  1045,
  4771,
  2026,
  19453,
  12339,
  1010,
  1998,
  2085,
  1045,
  1005,
  1049,
  2074,
  2667,
  2000,
  2131,
  2058,
  2009,
  1998,
  3564,
  2012,
  2188,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

We load a pretrained DistilBERT model.

In [ ]:
import torch
from transformers import AutoModel

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

We define a function to extract the last hidden state that summarizes the processed texts, and apply it to our training and test data sets.

In [ ]:
def extract_hidden_states(batch):
  inputs = {k:v.to(device) for k,v in batch.items()
  if k in tokenizer.model_input_names}
  with torch.no_grad():
    last_hidden_state = model(**inputs).last_hidden_state
  return{"hidden_state":last_hidden_state[:,0].cpu().numpy()}

In [ ]:
dataset_encoded.set_format("torch", columns = ["input_ids", "attention_mask", "label"])
dataset_hidden = dataset_encoded.map(extract_hidden_states, batched = True, batch_size= None)

As a next step, we train a simple classifier: Logistic Ridge Regression with Cross-Validation. We first store the hidden states and the labels of both the training and the test data set as X_train, y_train, X_test and y_test, respectively.

In [ ]:
import numpy as np

X_train = np.array(dataset_hidden["train"]["hidden_state"])
y_train = np.array(dataset_hidden["train"]["label"])
X_test = np.array(dataset_hidden["test"]["hidden_state"])
y_test = np.array(dataset_hidden["test"]["label"])

We fit the logistic ridge regresion model with cross validation via the fit() function and evaluate its accuracy in the training data set by applying the score() function.

In [ ]:
import sklearn
from sklearn.linear_model import RidgeClassifierCV

alphas = [1e-2,1,5,10,20,30]

lr_reg = RidgeClassifierCV(alphas = alphas, cv = 5)

lr_reg.fit(X_train, y_train)
lr_reg.score(X_train, y_train)
# 0.825

We get the tuned penalization parameter:

In [ ]:
lr_reg.alpha_

We also assess the accuracy in the test data set, which is somewhat lower:

In [ ]:
lr_reg.score(X_test, y_test)

We take a look at the regression coefficients:

In [ ]:
lr_reg.coef_

As a second approach, we fine-tune the pre-trained transformer model. As a first step, we split the original training set into a training X_train and a validation set X_val.

In [ ]:
X_train, X_val = dataset_encoded["train"].train_test_split(test_size = 0.2).values()

We take a look at both sets. X_train consists of 64 data points, X_val consists of 16 data points.

In [ ]:
X_train

In [ ]:
X_val

We define a classification model based on DistilBERT, which we have loaded earlier. In this model, we want to predict two different labels.

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 2
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels = num_labels)).to(device)

We set some hyperparameters for the training of the model. For details, see the main text.

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(X_val)// batch_size
model_name = "finetuned-dataset"
training_args = TrainingArguments(output_dir = model_name,
                                  num_train_epochs=6,
                                  weight_decay = 0.01,
                                  eval_strategy="epoch",
                                  logging_steps = logging_steps)

We define a function to get central performance metrics for our model, including the accuracy and the F1 score.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average = "weighted")
  acc = accuracy_score(labels, preds)
  return{"accuracy": acc, "f1": f1}

We train the model for 6 epochs, that is, six iterations through the training data. After each epoch, we obtain the accuracy and the F1 score in the validation set. Values close to 1 indicate a good prediction in the validation set.

In [ ]:
trainer = Trainer(model = model, args = training_args,
                  compute_metrics = compute_metrics,
                  train_dataset = X_train,
                  eval_dataset = X_val,
                  tokenizer = tokenizer)
trainer.train()

We are plotting the training and validation loss. Values close to 0 indicate good predictions in the training and validation set, respectively.

In [ ]:
import matplotlib.pyplot as plt

# Extracting training and evaluation loss values
train_loss = []
eval_loss = []

for entry in trainer.state.log_history:
    if 'epoch' in entry:
      if 'loss' in entry:
        if entry['epoch'] % 1 == 0:
            train_loss.append(entry['loss'])
      if 'eval_loss' in entry:
        eval_loss.append(entry['eval_loss'])

# Plotting the training and validation loss
plt.plot(train_loss, label='Training Loss')
plt.plot(eval_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

We can also extract and plot the accuracy and the F1 score after each epoch.

In [ ]:
# Extracting training and evaluation loss values
accuracy = []
f1_score = []

for entry in trainer.state.log_history:
  if 'eval_accuracy' in entry:
    accuracy.append(entry['eval_accuracy'])
  if 'eval_f1' in entry:
    f1_score.append(entry['eval_f1'])

# Plotting accuracy and F1 score:
plt.plot(accuracy, label='Accuracy in the Validation Set')
plt.plot(f1_score, label='F1 Score in the Validation Set')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Accuracy and F1 Score')
plt.legend()
plt.show()

As a next step, we want to calculate SHAP values for our fine-tuned model. We first define a pipeline, which directly transforms text into a label prediction. We call this pipeline pred:

In [ ]:
pred = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)

We demonstrate the use of pred in a simple example. The following cell gives the texts of the training data set:

In [ ]:
X_train["text"]

We now apply our pipeline pred to the first of these texts:

In [ ]:
X_train["text"][0]

In [ ]:
pred(X_train["text"][0])

Next, we install and import the shap package:

In [ ]:
#!pip install shap

In [ ]:
import shap

To prepare the calculation of the SHAP values, we define an explainer:

In [ ]:
explainer = shap.Explainer(pred)

We apply the explainer to two texts in our training set, which have the index 3 and 4. (In Python, these are indexed by 3:5: Text 3 is included, text 5 is not.)

In [ ]:
shap_values = explainer(X_train["text"][3:5])

We plot the SHAP values of these two texts:

In [ ]:
shap.plots.text(shap_values)

As the last step of our analysis, we want to calculate LIME values. We start with installing the LIME package:

In [ ]:
#!pip install lime

We import LimeTextExplainer, which is used to calculate the LIME values. For the plots, we also define class names, namely "Low Score" and "High Score". Finally, we define an explainer again, similar to the calculation of SHAP values.

In [ ]:
from lime.lime_text import LimeTextExplainer

class_names = ['Low Score', 'High Score']
explainer = LimeTextExplainer(class_names=class_names)

Following the example of the SHAP values, we define a pipeline named predictor that allows us to obtain predictions directly for a text. This functions needs the so-called softmax function, which is imported first:



In [ ]:
import torch.nn.functional as F

In [ ]:
def predictor(texts):
  outputs = model(**tokenizer(texts, return_tensors="pt", padding=True))
  probas = F.softmax(outputs.logits, dim=1).detach().numpy()
  return probas

We illustrate predictor with a simple example:

In [ ]:
predictor(X_train["text"][0])

Using the explainer we defined before, we calculate and plot the LIME values for a specific example, namely the fourth text of the training set. This text is indexed 3, since indices start with 0 in Python.

In this case, we calculate our LIME values based on 200 samples to keep the calculation concise. In practice, usually more samples are used.

In [ ]:
exp = explainer.explain_instance(X_train["text"][3], predictor, num_samples=200)


Finally, we plot the LIME values.

In [ ]:
exp.show_in_notebook(text=True)

# Julius idea:

Imagine a therapist and client talking:

In [ ]:
# Contentment levels for the client
contentment_levels = ["neutral", "neutral", "low", "low", "high", "low", "low", "high", "high", "high"]

# Therapist questions and expanded client responses
dialogue = [
    ("How have you been feeling lately?", 
     "I've been okay, just taking things one day at a time. It feels like life is on autopilot, where I go through the motions without much change. I wouldn’t say things are great, but they’re not terrible either. It’s been a balance between keeping busy and just trying to get through each day."),
    
    ("Can you tell me more about your current challenges?", 
     "Things have been a bit routine and monotonous. I wake up, do my work, and go to bed feeling like there’s not much joy in between. It’s like I’m just maintaining without really feeling connected to what I’m doing. Sometimes it feels like I'm stuck in a loop without excitement or motivation."),
    
    ("What thoughts are on your mind today?", 
     "Lately, I've been feeling pretty drained and unsure of what comes next. The uncertainty about where I’m headed is stressful. I worry that my efforts might not be enough or that I’m not making the right choices for my future. It's been difficult to shake off these feelings of doubt."),
    
    ("How do you usually cope when things are difficult?", 
     "Honestly, I haven’t been coping well. I find myself feeling overwhelmed and sometimes paralyzed by everything on my to-do list. When things get tough, I try to distract myself, but it doesn’t always help. Instead, I end up feeling like I’m avoiding what really matters, which only adds to my stress."),
    
    ("Can you describe a recent moment when you felt better?", 
     "Actually, yes. I had a nice evening with friends recently that really lifted my spirits. We laughed, shared stories, and just enjoyed each other’s company without any pressure. It reminded me of the value of connection and how even a few good hours can break up the tension I’ve been feeling."),
    
    ("What are your main sources of stress right now?", 
     "Work has been stressful, and I’m constantly worried about deadlines and keeping up with expectations. Sometimes it feels like there’s no end to the workload, and even when I finish something, there’s always more waiting. On top of that, personal responsibilities add another layer of pressure, making it hard to find balance."),
    
    ("What emotions are you experiencing as we talk?", 
     "I feel a bit anxious and unsure about what to do to feel better. There’s a sense of restlessness inside me that makes it difficult to fully relax. I’m aware that I need to make changes, but it feels overwhelming to even start. This combination of worry and hesitation is hard to shake."),
    
    ("What would help you feel more at ease?", 
     "Talking to friends or doing something fun tends to make me feel a lot better. It’s when I’m around people I trust that I can let my guard down and feel genuinely present. I think reconnecting with activities I enjoy, like hiking or music, would also be really grounding and uplifting."),
    
    ("Can you think of something that makes you feel grateful?", 
     "I'm grateful for the supportive friends I have. They really help me feel more positive and remind me that I’m not alone in facing challenges. Knowing there are people who care about me and are there when I need them is comforting. It makes even the harder days more bearable."),
    
    ("What plans or activities make you feel excited?", 
     "Thinking about an upcoming trip with my close friends makes me feel really happy and excited. It’s been a while since I’ve had something to look forward to, and the idea of exploring new places and making memories is energizing. It gives me hope and a break from the usual routine.")
]

# Print the dialog
for question, response in dialogue:
    print(f"Therapist: {question}")
    print(f"Client: {response}")
    print("\n")


## Call trained model to identify fluctuations of contentment during theraphy

models predict contentment (0 = low contentment and 1 = high contentment)

In [ ]:
# Extract only client responses
client_responses = [response for _, response in dialogue]
print("Client responses for extraction:")
print(len(client_responses))


# first:
print(pred(client_responses[0]))
# last: 
print(pred(client_responses[len(client_responses)-1]))

In [ ]:
# Actual contentment levels
contentment_levels = ["neutral", "neutral", "low", "low", "high", "low", "low", "high", "high", "high"]

# Predicted contentment levels based on model outputs
predicted_contentment_levels = []

# Iterate through each client response
for response in client_responses:
    prediction = pred(response)
    
    # Access the first (and only) item of the outer list
    if prediction and isinstance(prediction[0], list):
        prediction_list = prediction[0]
    else:
        prediction_list = prediction

    # Convert the prediction list into a dictionary for easier lookup
    prediction_dict = {p['label']: p['score'] for p in prediction_list}
    
    label_1_score = prediction_dict.get('LABEL_1', 0)
    label_0_score = prediction_dict.get('LABEL_0', 0)
    
    if label_1_score > 0.7:
        predicted_contentment = "high"
    elif label_0_score > 0.7:
        predicted_contentment = "low"
    else:
        predicted_contentment = "neutral"
    
    predicted_contentment_levels.append(predicted_contentment)

# Print actual and predicted contentment levels for comparison
for i in range(len(contentment_levels)):
    print(f"Response {i + 1}:")
    print(f"Actual: {contentment_levels[i]}")
    print(f"Predicted: {predicted_contentment_levels[i]}")
    print()

In [ ]:
import pandas as pd

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    "Response Number": range(1, len(contentment_levels) + 1),
    "Actual Contentment": contentment_levels,
    "Predicted Contentment": predicted_contentment_levels,
    "Match": [act == pred for act, pred in zip(contentment_levels, predicted_contentment_levels)]
})

# Print the comparison table
print(comparison_df)


# Calculate the percentage of matches
match_percentage = (comparison_df["Match"].mean()) * 100
# Print the match percentage
print(f"Percentage of matches: {match_percentage:.2f}%")

## Call sentiment model to identify fluctuations of contentment during theraphy


* Hugging Face model card: https://huggingface.co/siebert/sentiment-roberta-large-english
    + model predicts if a text is positive or negative
* Remark: if you want to avoid using "pipeline", see: https://github.com/chrsiebert/sentiment-roberta-large-english/blob/main/sentiment_roberta_prediction_example.ipynb

In [ ]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))

In [ ]:
# Predicted contentment levels based on model outputs
predicted_contentment_levels_sentiment = []

for idx, response in enumerate(client_responses):
    if contentment_levels[idx] != "neutral":
        prediction = sentiment_analysis(response)
        if prediction[0]['label'] == 'POSITIVE':
            predicted_contentment_levels_sentiment.append("high")
        elif prediction[0]['label'] == 'NEGATIVE':
            predicted_contentment_levels_sentiment.append("low")
    else:
        predicted_contentment_levels_sentiment.append(None)

In [ ]:
predicted_contentment_levels_sentiment

In [ ]:
import pandas as pd

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    "Response Number": range(1, len(contentment_levels) + 1),
    "Actual Contentment": contentment_levels,
    "Predicted Contentment": predicted_contentment_levels_sentiment,
    "Match": [act == pred for act, pred in zip(contentment_levels, predicted_contentment_levels_sentiment)]
})

# Print the comparison table
print(comparison_df)


# Calculate the percentage of matches
match_percentage = (comparison_df["Match"].mean()) * 100
# Print the match percentage
print(f"Percentage of matches: {match_percentage:.2f}%")